# fetch

In [ ]:
import ccxt
import pandas as pd
import os
import time

exchange = ccxt.hyperliquid()

markets = exchange.fetch_markets()

spot_pairs = [market['symbol'] for market in markets if market['type'] == 'spot']
spot_count = len(spot_pairs)

print(f"Hyperliquid spot pair num: {spot_count}")

folder_name = 'hyperliquid_spot'
os.makedirs(folder_name, exist_ok=True)


timeframe = '1d'
since = exchange.parse8601('2023-01-01T00:00:00Z')


for symbol in spot_pairs:
    all_ohlcv = []

   
    ohlcv = exchange.fetch_ohlcv(symbol, timeframe, since)
   
    if ohlcv:
        all_ohlcv.extend(ohlcv)

        ohlcv_df = pd.DataFrame(all_ohlcv, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
        ohlcv_df['timestamp'] = pd.to_datetime(ohlcv_df['timestamp'], unit='ms')

        file_name = os.path.join(folder_name, f"{symbol.replace('/', '_')}_ohlcv.csv")
        ohlcv_df.to_csv(file_name, index=False)

        print(f"Saved OHLCV data for {symbol} to {file_name}")
    else:
        print(f"No data available for {symbol}")

    print("Waiting for 10 seconds...")
    time.sleep(5)